In [4]:
# Interactive plots
%matplotlib notebook

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mypackage
import glob

np.set_printoptions(precision=4)
pd.options.display.max_rows = 4

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def load_spectra_db():
    def rename_database_group(substring, type_name):
        items = database[database["Type"].str.contains(substring)].index
        for i, idx in enumerate(items):
            database.loc[idx, "Type"] = f"{type_name}_{i}"
        
    file_name = "SpectralAbsorbtionDatabase.pkl"
    database = pd.read_pickle(file_name)
    database.reset_index(inplace=True)
    database.rename(columns={"index": "Type"}, errors="raise", inplace=True)
    
    # Remove the black plastic glove
    database = database[database["Type"].str.contains("black_latex_glove") != True]
    # Here the order does matter, since some substrings used are contained in more than one groups
    rename_database_group("_fat_", "Fat")
    rename_database_group("chicken_", "Chicken")
    
    rename_database_group("pu_belt", "Plastic_Pu_Belt")
    rename_database_group("blue_belt_roll", "Plastic_Belt")
    rename_database_group("tube", "Plastic_Tube")
    rename_database_group("glove", "Plastic_Glove")
    rename_database_group("liner", "Plastic_Liner")
    rename_database_group("belt", "Plastic_Belt")
    types = database[database["Type"].str.contains("Plastic")]["Type"].str.replace("[0-9()]+$", "")
    plastic_types = dict(enumerate(np.unique(types)))
    
#     plastics = database[database["Type"].str.contains("plastic") &  (database["Type"].str.contains("liner") != True)]
    return database
database = load_spectra_db()
database

,Type,800.43,800.68,800.93,801.17,801.42,801.67,801.92,802.17,802.41,...,2478.6,2481.0,2483.3,2485.7,2488.1,2490.5,2492.9,2495.3,2497.7,2500.1
0,Plastic_Tube_0,1.38810,1.38980,1.41220,1.43610,1.45510,1.41660,1.38990,1.36990,1.38080,...,1.7836,1.7364,1.7186,1.7332,1.7216,1.70260,1.72400,1.76070,1.75660,1.74770
1,Fat_0,0.80156,0.81471,0.82090,0.81225,0.79933,0.79466,0.79840,0.80355,0.81272,...,1.8789,1.9289,1.9613,1.9575,1.9609,1.92980,1.86710,1.85500,1.83040,1.84770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Plastic_Liner_7,0.82905,0.81139,0.79338,0.79069,0.80060,0.80919,0.81061,0.80750,0.81187,...,1.0943,1.1002,1.1123,1.1064,1.0643,0.99794,0.94007,0.91045,0.90163,0.89998
73,Plastic_Liner_8,0.28886,0.28498,0.28099,0.28096,0.28322,0.28221,0.27875,0.27903,0.28404,...,1.3783,1.3591,1.3683,1.3922,1.3812,1.33500,1.29670,1.26240,1.23240,1.24080


(array(['Plastic_Belt_', 'Plastic_Glove_', 'Plastic_Liner_',
        'Plastic_Pu_Belt_', 'Plastic_Tube_'], dtype=object),
 {0: 'Plastic_Belt_',
  1: 'Plastic_Glove_',
  2: 'Plastic_Liner_',
  3: 'Plastic_Pu_Belt_',
  4: 'Plastic_Tube_'})

### TODO: Use the labels for the TOMRA dataset as the labels for the fake dataset and just fill in the image

In [6]:
labels_path = "../data/tomra/"
_, labels, _ = mypackage.Dataset.load(labels_path, only_with_contaminant=True) # load_labels(labels_path)

In [7]:
labels.shape, np.unique(labels, return_counts=True), np.unique(labels[0], return_counts=True)

((24, 100, 100, 1),
 (array([1, 2, 3], dtype=int32), array([200211,  36685,   3104])),
 (array([1, 2, 3], dtype=int32), array([8587, 1303,  110])))

In [13]:
def generate_image(base_label):
    ''''Generates fake images based in the initialized FakeDataset parameters

        # Where x is the generated image and y is the corresponding label
        returns x, y'''
    # TODO: Fill the x image with randomly selected spectra (with replacement) based on the label
    #             np.random.choice(spectras_of_type, pixels_needed, replace=True)
    y = base_label
    x = 0# TODO:...
    return x, y

def get_images(numb_images):
    X, Y = [], []
    numb_available_labels = len(labels) # 
    selected_labels = np.random.choice(numb_available_labels, numb_images, replace=False)
    for i in selected_labels:
        x, y = generate_image(labels[i])
        X.append(x)
        Y.append(y)
    return np.array(X), np.array(Y)

X, Y = get_images(numb_images=1);

In [16]:
type(database.columns[1])

float

In [39]:
fake_data = mypackage.FakeDataset()
fake_data.get_images(1)

(8580,)
1


IndexError: boolean index did not match indexed array along dimension 2; dimension is 2203 but corresponding boolean dimension is 1

In [38]:
x = np.zeros((100, 100, 208))
y = labels[0]
np.unique(y, return_counts=True)

(array([1, 2, 3], dtype=int32), array([8587, 1303,  110]))

In [49]:
x[y==1]

IndexError: too many indices for array